## ALR Client Side

This notebook is a copy similar to ALR_Client_Side found in:
https://github.com/rfernand387/ALR_Earth_Engine/blob/master/ALR_Client_Side.ipynb

The input image is assumed to be in the format generated by SL2P10_control.ipynb:
https://github.com/kateharvey/Sentinel2_ALR/blob/main/shared/SL2P10_control.ipynb

In [15]:
import ee
import scipy
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as skl ; from sklearn import preprocessing ; from sklearn import linear_model
import tensorflow as tf
import json
from collections import OrderedDict
import time
import math
import csv
import os
import geemap

# import custom module
import ALR_functions as alr


In [3]:

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWgcZbiUNxX4_LKqVpJe4ycpo1SuEATp-UCFczD24mDWE4kZoEaxgXw



Successfully saved authorization token.


Currently the trimmed data is processed then in a neural network created using tensorflow to find nonlinear relationships between the predictor and the response. Earth Engine does not have this functionality (for free) to generate neural network based models.

Here we also see how the server side in the Earth Engine API is completely separate from the client side on the local machine. We need
to export our trimmed data as a CSV to a google drive which is synced into the "gdrive" folder in our local machine using the 
Backup and Sync software or using google-drive-ocamlfuse on Linux

For a more robust way to check if the data has been exported properly, we can use a wait loop to check on our local machine until the exported data file exists in the synced "gdrive" folder

From here on out, all of the processing is done using your local hardware and packages, so it may be helpful to use a powerful machine.

In [19]:
def parse_layer(feature):
    feature = ee.Feature(feature)
    prev_layer_size = feature.getNumber("prev_layer_size")
    num_nodes = feature.getNumber("num_nodes")
    node_size = prev_layer_size.subtract(1)
    activation = feature.getString("activation")
    
    node_collection = ee.ImageCollection(ee.List.sequence(1, num_nodes)\
                        .map(lambda node: ee.ImageCollection(ee.List.sequence(ee.Number(node).toInt(), ee.Number(node)\
                                    .toInt().add(node_size.multiply(num_nodes)), num_nodes)\
                        .map(lambda index: ee.Image(feature.getNumber(ee.Number(index).toInt())))).toBands()\
                        .set({"bias": feature.getNumber(ee.Number(node).toInt().add(prev_layer_size.multiply(num_nodes)))})))
    
    return ee.List([node_collection, activation])

In [ ]:
# parse neural net from gee asset
nnet = ee.FeatureCollection("users/rfernand387/nnet")
nnet_inputs = nnet.filter(ee.Filter.eq("layer_num", 0)).first()
num_inputs = nnet_inputs.getNumber("num_nodes")
nnet = nnet.filterBounds(ee.Geometry.Point([0,0]))
layer_list = nnet.sort("layer_num").toList(nnet.size())
neural_net = layer_list.map(parse_layer)



In [ ]:
# define input image for network and apply network
# make sure input bands match inputs when network was trained, including any scaling
select_features = ['MSR', 'GVI', 'RDVI', 'SGI', 'EVI2']
nnet_inputs = inputImage.select(select_features)
prediction_data = ee.Image(neural_net.iterate(apply_nnet, nnet_inputs)).rename("NNET")

In [25]:
# display result using geemap
Map = geemap.Map(center=[40,-100], zoom=4)
Map
Map.addLayer(prediction_data)

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [27]:
Map.addLayer(nnet_inputs )


EEException: Image.select: Pattern 'linear' did not match any bands.